## No distillation



In this notebook we train models without applying any distillation or transfer learning technique.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from google.colab import files
uploaded = files.upload()
!unzip dataset.zip -d dataset

In [0]:
import warnings
import os
import shutil
import glob
import random
import random
from fastai.vision import *

warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

dataset="dataset"
classesPaths=sorted(glob.glob(dataset+'/*'))
classes=[pt.split(os.sep)[-1] for pt in classesPaths if os.path.isdir(pt)]
images=[pt for pt in classesPaths if not os.path.isdir(pt)]

os.makedirs(dataset+'/train')
os.makedirs(dataset+'/valid')
os.makedirs(dataset+'/images')

for im in images:
  shutil.move(im,dataset+'/images/')

for cl in classes:
  os.mkdir(dataset+'/train/'+cl)
  images=sorted(glob.glob(dataset+'/'+cl+'/*'))
  for i in range(int(len(images)*0.75)):
    images=sorted(glob.glob(dataset+'/'+cl+'/*'))
    j=random.randint(0,len(images)-1)
    shutil.move(images[j],dataset+'/train/'+cl)
  os.mkdir(dataset+'/valid/'+cl)
  images=sorted(glob.glob(dataset+'/'+cl+'/*'))
  for i in range(len(images)):
    shutil.move(images[i],dataset+'/valid/'+cl)

def learn_with_model(dataset,model):
  data=ImageDataBunch.from_folder(dataset,
        ds_tfms=get_transforms(), size=224,bs=32).normalize(imagenet_stats)
  learn = cnn_learner(data, model, metrics=accuracy)
  learn.fit_one_cycle(2)
  learn.unfreeze()
  learn.lr_find()
  lr=learn.recorder.lrs[np.argmin(learn.recorder.losses)]
  if lr<1e-05:
    lr=1e-03
  learn.fit_one_cycle(8,max_lr=slice(lr/100,lr))
  return learn,data

def learn_best_model(dataset,models):
  best_accuracy=0
  for model in models:
    learn,data=learn_with_model(dataset,model)
    acc=learn.validate()[1].item()
    if acc>best_accuracy:
      best_model=learn
      best_accuracy=acc
  return best_model,data

learner,data=learn_best_model(dataset,[models.resnet50,models.resnet34,models.resnet101])
learner.export('/content/drive/My Drive/learner_ND.pkl')
